### Item 24: Consider variance for generic types

#### variance(변성)
상속 관계를 가지는 `T`, `U` 타입에 대해, `Container<T>`,`Container<U>` 타입에 대해서도 업캐스를 사용하고 싶다면, variance 제약이 필요하다.

variance modifier 를 사용해서 variance 제약을 만들 수 있다. 

**variance modifier**
- `in T`: 이 타입의 클래스는 멤버 함수가 `T` 타입 값을 parameter 로만 받고, T 타입 값을 반환하지 않는다.
- `out T`: 이 타입의 클래스는 멤버 함수가 `T` 타입 값을 반환하기만 하고, T 타입 값을 parameter 로 받지 않는다.

In [1]:
class Box<T>(private var contents: T) {
    fun put(item: T) { contents = item }
    fun get(): T = contents
}

class InBox<in T>(private var contents: T) {
    fun put(item: T) { contents = item }
//    fun get(): T = contents // ERROR
}

class OutBox<out T>(private var contents: T) {
//    fun put(item: T) { contents = item } // ERROR
    fun get(): T = contents
}

![variance](./24_variance.png)

**covariance(공변)**
- `Box<T>` 는 invariance(무공변) 이다. 즉, `Box<Cat>` 과 `Box<Pet>` 사이에 아무런 관계가 없다. 어느 쪽도 반대쪽에 대입될 수 없다.
- `OutBox<out T>` 는 covariance(공변) 이다. 즉, `Box<Cat>` 과 `Box<Pet>` 의 업캐스트 방향이 `Cat` 과 `Pet` 과 동일하다.
- `InBox<in T>` 는 contravariance(반공변) 이다. 즉, `Box<Pet>` 과 `Box<Cat>` 의 업캐스트 방향이 `Cat` 과 `Pet` 과 반대이다.

#### Function type
Kotlin function type 은 parameter type 에 대해선 contravariant 이며, return type 에 대해선 covariant 이다.

예시로, function type `(Cat) -> Pet` 은
parameter type 에 대해선 contravariant 하다. 즉, `(Pet) -> Pet` 을 대입할 수 있다.
- return type 에 대해선 variant 하다. 즉, `(Cat) -> Cat` 을 대입할 수 있다.

#### The safety of variance modifiers(variance 한정자의 안정성)

Kotlin 에선 `Array` 는 invariant 이다.

`Array<Int>` 타입을 `Array<Any>` 타입으로 업캐스팅하고, `String` 타입을 할당하려고 하면 이는 결함이다.

In [1]:
class OutBox<out T> {
    private var value: T? = null // private 으로 선언하면 'in' 위치에 out T 타입을 할당할 수 있다고 책에선 설명했으나,
    
    fun set(value: T) { this.value = value } // ERROR 가 발생한다. 버전 확인이 필요하다.
    fun get(): T = this.value ?: error("value not set")
}

Line_1.jupyter.kts (8:20 - 21) Type parameter T is declared as 'out' but occurs in 'in' position in type T

#### Variance modifier 위치

1. class/interface 의 선언부

class/interface  가 사용되는 모든 곳에 영향을 미친다.

In [2]:
class Box<out T>(private var contents: T) {}
val boxStr: Box<String> = Box("Str")
val boxAny: Box<Any> = boxStr

2. class/interface 를 사용하는 곳

사용되는 모든 곳이 아닌 일부에서만 variance modifier 를 적용하고 싶을 때 사용한다.

In [3]:
class Box<T>(private var contents: T) {}
val boxStr: Box<String> = Box("Str")
val boxAny: Box<out Any> = boxStr

두 번째 케이스의 대표적인 예시가 `MutableList`

`in` modifier 를 선언부에 사용했다면, element 를 return 할 수 없다.

하지만 사용처에서 `in` 을 추가함으로써, 여러 타입을 add 할 수 있다.

아래 코드는 함수에서 `MutableList<in Puppy>` 을 사용함으로써, `MutableList<Dog>`, `MutableList<Cute>` 타입에 `Puppy` 타입을 add 한다.  

In [5]:
interface Dog
interface Cutie
data class Puppy(val name: String): Dog, Cutie
data class Hound(val name: String): Dog
data class Cat(val name: String): Cutie

fun fillWithPuppies(list: MutableList<in Puppy>) {
    list.add(Puppy("Jim"))
    list.add(Puppy("Beam"))
}

val dogs = mutableListOf<Dog>(Hound("Pluto"))
fillWithPuppies(dogs)
println(dogs)

val animals = mutableListOf<Cutie>(Cat("Felix"))
fillWithPuppies(animals)
println(animals)

[Hound(name=Pluto), Puppy(name=Jim), Puppy(name=Beam)]
[Cat(name=Felix), Puppy(name=Jim), Puppy(name=Beam)]


#### 참고
- List, Set, Map.value 의 type parameter 는 covariant 이다.
- Array, MutableList, MutableSet, MutableMap 의 type parameter 는 invariant 이다. 